## Clase 2 – Configuraciones Iniciales e Instalación del cliente.
Continuando con la creación de una arquitectura ‘cliente-servidor’, en esta clase:
- Configurar los parámetros básicos de nuestro servidor Linux.
- Instalar el cliente desde el cual accederemos a nuestro servidor Linux.
- Manejo de los comandos básicos de gestión de archivos en Linux.
- Manejo de permisos de archivos y directorios en Linux

### 1 Configuración del Servidor Linux.
Para usar propiamente nuestra VM Ubuntu como un servidor Linux, debemos configurarle una dirección IP estática. Estos son los pasos:

1.1) Desde Virtualbox, inicie la VM TelematicaAdmRedes que creamos en la clase anterior e introduzca los datos de usuario y clave.

1.2) Vamos a escalar privilegios convirtiéndonos administrador dentro del sistema. En Linux, el administrador es llamado el usuario ‘root’. 

In [ ]:
carlosfranz@telematicaadmredes:~$ sudo -i
[sudo] password for carlosfranz:
root@telematicaadmredes:~# whoami
root

Y verificamos con ‘whoami’ que realmente seamos usuario ‘root’.

1.3) Vamos a consultar nuestras configuraciones actuales con el comando ‘ifconfig’. Alli se muestra las interfaces de red instaladas. Nos interesa ‘enp0s3’ que es la tarjeta de red de nuestro servidor. Su dirección IP actual (192.168.1.9) asignada dinámicamente y que queremos cambiar por una fija y la máscara de subred (255.255.255.0).

In [ ]:
carlosfranz@telematicaadmredes:~$ ifconfig
enp0s3 Link encap:Ethernet HWaddr 08:00:27:8b:f0:53
 inet addr:192.168.1.9 Bcast:192.168.1.255
Mask:255.255.255.0
inet6 addr: fe80::a00:27ff:fe8b:f053/64 Scope:Link
 UP BROADCAST RUNNING MULTICAST MTU:1500 Metric:1
 RX packets:79 errors:0 dropped:0 overruns:0 frame:0
 TX packets:68 errors:0 dropped:0 overruns:0
carrier:0
 collisions:0 txqueuelen:1000
 RX bytes:10662 (10.6 KB) TX bytes:10934 (10.9 KB)
lo Link encap:Local Loopback
 inet addr:127.0.0.1 Mask:255.0.0.0
 inet6 addr: ::1/128 Scope:Host
 UP LOOPBACK RUNNING MTU:65536 Metric:1
 RX packets:160 errors:0 dropped:0 overruns:0
frame:0
 TX packets:160 errors:0 dropped:0 overruns:0
carrier:0
 collisions:0 txqueuelen:1
 RX bytes:11840 (11.8 KB) TX bytes:11840 (11.8 KB)


1.4) Consultamos la Puerta de Enlace actual con el comando ‘ip route show’. El parámetro ‘default’ nos muestra que es la dirección:’192.168.1.1’.

In [ ]:
root@telematicaadmredes:/etc/network# ip route show
default via 192.168.1.1 dev enp0s3
192.168.1.0/24 dev enp0s3 proto kernel scope link src
192.168.1.9 

1.5) Vamos a usar el comando Linux ‘pwd’ (present working directory) para saber en que directorio nos encontramos en este momento.

In [ ]:
root@telematicaadmredes:~# pwd
/root

1.6) Con el comando ‘cd’ (change directory) nos iremos a la ruta donde se encuentran los archivos de configuración, concretamente las configuraciones de red: etc/network. En Linux, el directorio donde se guardan las configuraciones del sistema es: /etc

In [ ]:
root@telematicaadmredes:~# cd /etc/network/
root@telematicaadmredes:/etc/network# ls
if-down.d if-post-down.d if-pre-up.d if-up.d interfaces
interfaces.d 

1.7) Allí está el archivo ‘interfaces’. Vamos a respaldarlo (con cp) y a abrirlo con el
editor ‘nano’.

In [ ]:
root@telematicaadmredes:/etc/network# cp interfaces
interfaces.dist
root@telematicaadmredes:/etc/network# nano interfaces
# This file describes the network interfaces available on
your system
# and how to activate them. For more information, see
interfaces(5).
source /etc/network/interfaces.d/*
# The loopback network interface
auto lo
iface lo inet loopback
# The primary network interface
auto enp0s3
iface enp0s3 inet dhcp

En ‘interfaces’ se muestran dos adaptadores de red: lo (https://es.wikipedia.org/wiki/Loopback) y enp0s3. De acuerdo a esta configuración, a enp0s3 le es asignado dinámicamente una dirección a través de DHCP.

Nosotros vamos a fijar una dirección a la interfaz enp0s3 colocando los datos como el cuadro:

In [ ]:
# This file describes the network interfaces available on
your system
# and how to activate them. For more information, see
interfaces(5).
source /etc/network/interfaces.d/*
# The loopback network interface
auto lo
iface lo inet loopback
# The primary network interface
auto enp0s3
iface enp0s3 inet static
address <1>
netmask <2>
gateway <3>
dns-nameservers 8.8.8.8 8.8.4.4

Siendo <1> la dirección IP deseada para nuestro servidor. 

En mi caso es ‘192.168.1.7’. <2> la máscara de subred, usaremos la encontrada en el paso 2.3. <3> usaremos la puerta de enlace encontrado en el paso 2.4. También podemos fijar desde aquí los nombres de nuestro servidor de dominio, usaremos los de Google (8.8.8.8 8.8.4.4).

1.8) Para que los cambios hagan efecto, debemos reiniciar el servicio ‘networking’ de nuestro servidor.

In [ ]:
root@telematicaadmredes:/etc/network# service networking
restart

1.9) Verificamos los cambios.

In [ ]:
root@telematicaadmredes:/etc/network# ifconfig
enp0s3 Link encap:Ethernet HWaddr 08:00:27:8b:f0:53
 inet addr:192.168.1.9 Bcast:192.168.1.255
Mask:255.255.255.0
 inet6 addr: fe80::a00:27ff:fe8b:f053/64 Scope:Link
 UP BROADCAST RUNNING MULTICAST MTU:1500 Metric:1
 RX packets:748 errors:0 dropped:0 overruns:0
frame:0
 TX packets:410 errors:0 dropped:0 overruns:0
carrier:0
 collisions:0 txqueuelen:1000
 RX bytes:65140 (65.1 KB) TX bytes:52288 (52.2 KB)
lo Link encap:Local Loopback
 inet addr:127.0.0.1 Mask:255.0.0.0
 inet6 addr: ::1/128 Scope:Host
 UP LOOPBACK RUNNING MTU:65536 Metric:1
 RX packets:160 errors:0 dropped:0 overruns:0
frame:0
 TX packets:160 errors:0 dropped:0 overruns:0
carrier:0
 collisions:0 txqueuelen:1
 RX bytes:11840 (11.8 KB) TX bytes:11840 (11.8 KB)

### 2 Instalación del cliente

Un programa cliente ssh es necesario para acceder a nuestro servidor. Si el terminal desde el que deseamos acceder a nuestro servidor es Linux, la consola puede ser la vía para hacerlo. Pero no siempre tendremos permisos para acceder desde consola o necesitaremos entrar desde un terminal Windows, Mac o Android.

Instalaremos PuTTY. Si están en Windows descarguen los instaladores desde aquí: Página de descarga de PuTTY

Si están en Linux, busquen en sus repositorios locales y el gestor de paquetes lo instalara.

Se encontrarán con esta ventana, que por defecto marca el protocolo de conexión:SSH y el puerto:22. Solo falta que tipeen la dirección de nuestro servidor Linux, la que configuramos como estática y les abrirá la consola. Esto lo pueden hacer desde cualquier terminal que este en la misma red que nuestro servidor. 

![alt text](./images/Screenshot from 2022-03-09 15-41-05.png)


### 3 Manejo de archivos

Entre a su servidor Linux (directamente o via ssh).

#### Creación de Directorios

La creación de directorios es un proceso realmente sencillo. Para esto, el sistema nos provee una herramienta llamada mkdir. Su forma es la siguiente:

In [ ]:
mkdir [ruta] [directorio]

#### Creación de archivos

La forma más fácil y directa de crear un archivo vacío en el sistema es con el comando ‘touch’.

In [ ]:
touch [opciones] [ruta][nombre_del_archivo]

#### Copiando archivos

La copia de archivos desde línea de comandos cp y tiene esta estructura básica

In [ ]:
cp [opciones] [archivo] [destino]

Este comando fue utilizado la clase pasada para la creación del archivo de respaldo de /etc/network/interfaces. Entonces llegamos a la ruta ‘/etc/network’ e hicimos esto:

In [ ]:
cp interfaces interfaces.dist

Pudimos desde cualquier otra ruta ejecutar lo anterior así:

In [ ]:
cp /etc/network/interfaces /etc/network/interfaces.dist

Con cp está permitido el uso de comodines (wildcards). Por ejemplo, queremos hacer un respaldo de todos los archivos de configuración del sistema (ubicados, como vimos en la clase anterior en /etc). Primero creamos un directorio /home/MisBackups.

In [ ]:
mkdir /home/MisBackups

Luego

In [ ]:
cp -R /etc/* /home/MisBackups

Para esto, utilizamos la opcion ‘-R’ que indica al cp de copiar directorios y su contenido de forma recursiva.

Consulte la ayuda para conocer todas las variantes del comando cp

In [ ]:
cp --help

#### Borrando archivos

El borrado de archivos esta implementado con el comando rm.

In [ ]:
rm [opciones] [ruta] [archivo]

### 4 Permisos de acceso
Cada archivo de nuestro sistema de archivos (incluso los directorios) tiene determinados permisos de acceso. Estos son, básicamente tres: Lectura (r), Escritura (w) y Ejecución (x). Estos permisos pueden ser aplicados al usuario dueño del archivo, al grupo de usuarios al cual pertenece y a todos los demás usuarios del sistema. Esto significa que podemos asignar permisos de ejecución al dueño de un archivo, pero podemos asignar permisos de sólo lectura a todos los demás usuarios.

Para listar los archivos y sus respectivos permisos, usaremos el comando ‘ls’

In [ ]:
ls –l /home/MisBackups

Que devolverá un listado como el siguiente:

In [ ]:
-rw-r--r-- 1 carlosfranz users 12288 Jun 27 05:47 aliases.db
-rw-r--r-- 1 carlosfranz users 279 Jun 27 05:47 alsa-pulse.conf
drwxr-xr-x 2 carlosfranz users 4096 Jun 27 05:47 alternatives
drwxr-xr-x 10 carlosfranz users 4096 Jun 27 05:47 apache2
drwxr-xr-x 2 carlosfranz users 126 Jun 27 05:47 apparmor
drwxr-xr-x 7 carlosfranz users 4096 Jun 27 05:47 apparmor.d
-rw-r--r-- 1 carlosfranz users 769 Jun 27 05:47 appstream.conf
-rw-r--r-- 1 carlosfranz users 123 Jun 27 05:47 apulse.conf
-rw-r--r-- 1 carlosfranz users 263 Jun 27 05:47 asound-pulse.conf
drwxr-x--- 2 carlosfranz users 6 Jun 27 05:47 audisp
drwxr-x--- 2 carlosfranz users 6 Jun 27 05:47 audit
-rw-r--r-- 1 carlosfranz users 14356 Jun 27 05:47 autofs.conf
-rw-r--r-- 1 carlosfranz users 798 Jun 27 05:47 auto.master
drwxr-xr-x 2 carlosfranz users 6 Jun 27 05:47 auto.master.d
-rw-r--r-- 1 carlosfranz users 524 Jun 27 05:47 auto.misc

La primera columna de este listado nos indica el permiso asignado, en función a la letra. Si el primer carácter tiene una letra (‘d’) es un directorio, sino aparecerá un (‘-’). Para el resto de los carácteres, los dividiremos en grupos de tres:

- Carácteres 1-2-3: son permisos asignados al usuario
- Carácteres 4-5-6: son permisos asignados al grupo
- Carácteres 7-8-9: son permisos asignados a todos los demás usuarios.

si en lugar de una letra aparece un símbolo menos (‘-’) quiere decir que el permiso no está asignado.

Ejemplos:
|            |                                                                                                                                          |
|------------|------------------------------------------------------------------------------------------------------------------------------------------|
| -rw-rw-r-- | Es un archivo que puede ser leído por todos los usuarios del sistema, pero solo puede ser modificado por su propietario y por su grupo   |
| drwxrwxrwx | Se trata de un directorio con permisos de lectura, escritura y ejecución para el dueño, el grupo y el resto de los usuarios del sistema. |
| -r—r—r--   | Es un archivo que podría ser leído por todos los usuarios, pero no modificado ni ejecutado por ninguno.                                  |
| -rwx------ | Es un archivo que solo puede ser leído, modificado y ejecutado por su dueño.                                                             |

#### Gestión de permisos.

Para manejar permisos Linux nos ofrece una herramienta llamada chmod.

In [ ]:
chmod [quien] [+/-] [permiso] [archivo]

El parámetro:
 
- [quien]: puede contener una letra u (usuario dueño del archivo), una letra g (grupo) o una letra o (todos los demás).
- [+/-]: permite asigna un permiso con el carácter ‘+’ o denegarlo con ‘-’.
- [permiso]:x, r, w vistos anteriormente.

Ejemplos:
|                           |                                                                                                                                                 |
|---------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------|
| chmod u+x appstream.conf  | Asigna permiso de ejecución del archivo appstream.conf solo para su dueño.                                                                      |
| chmod o-x movebackup.py   | Deniega permiso de ejecución del archivo movebackup.py a todos los usuarios que no pertenezcan al grupo del usuario propietario de ese archivo. |
| chmod ug+r appstream.conf | Asigna permiso de lectura del archivo appstream.conf para su dueño y para el grupo al que pertenece                                             |